<a href="https://colab.research.google.com/github/XingxinHE/Python-for-Data-Analysis-Notes/blob/master/8_3_Reshaping_and_Pivoting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

## Reshaping with Hierarchical Indexing

.stack()

.unstack()

In [2]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
          index=pd.Index(['Ohio', 'Colorado'], name='state'),
          columns=pd.Index(['one', 'two', 'three'],
          name='number'))

data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [4]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [5]:
result.unstack() #默认unstack的level是-1，也就是最里面的那一层

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [7]:
result.unstack(level=0) #可以specify是表层

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [8]:
result.unstack('state') #也可以specify层的名字，unstack什么，什么就是columns

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


unstack可能会产生出NaN，但是stack的默认设置是忽略NaN

In [9]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [11]:
data2.unstack() #可以看到有几个NaN

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [12]:
data2.unstack().stack() #默认是清除NaN的

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [13]:
data2.unstack().stack(dropna=False) #可以specify不要drop na

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

当你unstack某一Level变成columns，上面提到了这句话，但是还要加一句，变成columns之后是变成最低级的columns，也就是bottom的columns

In [14]:
df = pd.DataFrame({'left': result, 'right': result + 5},
          columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [15]:
df.unstack('state')
#可以观察到state被拆开了，
#而且位于column下面的位置

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [16]:
df.unstack('state').stack('side') #可以specifystack哪一层

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

In [17]:
df.unstack('state').stack('state')

side             left  right
number state                
one    Ohio         0      5
       Colorado     3      8
two    Ohio         1      6
       Colorado     4      9
three  Ohio         2      7
       Colorado     5     10

## Pivoting "Long" to "Wide" Format

多个时间序列数据通常是以所谓的“长格式”（long）或“堆叠格式”（stacked）存储在数据库和CSV中的

🌟“长”是指往下，row越来越多

🌟“宽”是指往右，columns越来越多

In [21]:
url = 'https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/macrodata.csv'
data = pd.read_csv(url)
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [22]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [23]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
columns

Index(['realgdp', 'infl', 'unemp'], dtype='object', name='item')

In [24]:
data = data.reindex(columns=columns) #reindex这个很6，能把想要的要了，其它不要
data.head()

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2


In [25]:
#处理好columns之后，再处理index（row）
data.index = periods.to_timestamp('D', 'end')

In [26]:
data.head()

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2


In [27]:
ldata = data.stack()
ldata.head()

date                           item   
1959-03-31 23:59:59.999999999  realgdp    2710.349
                               infl          0.000
                               unemp         5.800
1959-06-30 23:59:59.999999999  realgdp    2778.801
                               infl          2.340
dtype: float64

In [28]:
ldata = ldata.reset_index()
ldata.head()

,date,item,0
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340


In [29]:
ldata = ldata.rename(columns={0:'value'})
ldata.head()

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340


这种格式是MySQL的经典格式，在于两个关键信息，date和item是关键信息，只要有这两个信息，value自然而然就可以得出。

那么如果你习惯dataframe格式的话，就可以用pivot来实现

In [31]:
pivoted = ldata.pivot('date', 'item', 'value') #把它从long format变成wide format
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


上面只有一个value，那假如有两个value呢？

In [33]:
ldata['value2'] = np.random.randn(len(ldata))
ldata.head()

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-1.188260
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.142315
2,1959-03-31 23:59:59.999999999,unemp,5.800,0.335999
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,1.924186
4,1959-06-30 23:59:59.999999999,infl,2.340,-0.118000


In [35]:
pivoted = ldata.pivot('date', 'item')
pivoted
#那么拆出来的值就会变成hierarchical 了

value             ...    value2          
item                           infl    realgdp  ...   realgdp     unemp
date                                            ...                    
1959-03-31 23:59:59.999999999  0.00   2710.349  ... -1.188260  0.335999
1959-06-30 23:59:59.999999999  2.34   2778.801  ...  1.924186 -0.220719
1959-09-30 23:59:59.999999999  2.74   2775.488  ...  0.506217  3.351093
1959-12-31 23:59:59.999999999  0.27   2785.204  ... -0.853514  0.383990
1960-03-31 23:59:59.999999999  2.31   2847.699  ...  0.383475  0.960597
...                             ...        ...  ...       ...       ...
2008-09-30 23:59:59.999999999 -3.16  13324.600  ... -2.590649 -1.297989
2008-12-31 23:59:59.999999999 -8.79  13141.920  ... -0.716232  2.218971
2009-03-31 23:59:59.999999999  0.94  12925.410  ... -0.699466 -0.833255
2009-06-30 23:59:59.999999999  3.37  12901.504  ...  0.142441  0.980879
2009-09-30 23:59:59.999999999  3.56  12990.341  ...  1.055907 -0.883785

[203 rows x 6 columns]

In [36]:
pivoted['value'][:5] #用这种方法可以只选其中一个value

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


pivot等效于啥？pivot = set_index+unstack

In [37]:
unstacked = ldata.set_index(['date', 'item'])
unstacked.head()

value    value2
date                          item                       
1959-03-31 23:59:59.999999999 realgdp  2710.349 -1.188260
                              infl        0.000 -0.142315
                              unemp       5.800  0.335999
1959-06-30 23:59:59.999999999 realgdp  2778.801  1.924186
                              infl        2.340 -0.118000

In [38]:
unstacked = unstacked.unstack('item')
unstacked.head()

value            ...    value2          
item                           infl   realgdp  ...   realgdp     unemp
date                                           ...                    
1959-03-31 23:59:59.999999999  0.00  2710.349  ... -1.188260  0.335999
1959-06-30 23:59:59.999999999  2.34  2778.801  ...  1.924186 -0.220719
1959-09-30 23:59:59.999999999  2.74  2775.488  ...  0.506217  3.351093
1959-12-31 23:59:59.999999999  0.27  2785.204  ... -0.853514  0.383990
1960-03-31 23:59:59.999999999  2.31  2847.699  ...  0.383475  0.960597

[5 rows x 6 columns]